# 4.6 学習と検証の実施

- 本ファイルでは、OpenPoseの学習と検証の実施を行います。AWSのGPUマシンで計算します。
- p2.xlargeで45分ほどかかります。


# 学習目標

1.	OpenPoseの学習を実装できるようになる

# 事前準備

- これまでの章で実装したクラスと関数をフォルダ「utils」内に用意しています


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/packages")

In [20]:
cd "/content/data"

/content/data


In [16]:
import os
import urllib.request
import zipfile
import tarfile
# フォルダ「data」が存在しない場合は作成する
data_dir = "/content/data"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

In [17]:
# MSCOCOの2014 Val images [41K/6GB]をダウンロード
# 6GBのダウンロードと解凍なので時間がかかります（10分弱）
url =  "http://images.cocodataset.org/zips/val2014.zip"
target_path = os.path.join(data_dir, "val2014.zip") 

if not os.path.exists(target_path):
    urllib.request.urlretrieve(url, target_path)
    
    zip = zipfile.ZipFile(target_path)
    zip.extractall(data_dir)  # ZIPを解凍
    zip.close()  # ZIPファイルをクローズ

In [21]:
!wget https://www.dropbox.com/s/bd9ty7b4fqd5ebf/mask.tar.gz

--2021-03-02 16:59:14--  https://www.dropbox.com/s/bd9ty7b4fqd5ebf/mask.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/bd9ty7b4fqd5ebf/mask.tar.gz [following]
--2021-03-02 16:59:15--  https://www.dropbox.com/s/raw/bd9ty7b4fqd5ebf/mask.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf155bc8507c1c74e64add2c4df.dl.dropboxusercontent.com/cd/0/inline/BJ4FDzgPHcKz-Y5dVuMhX3RMv21LHZDbg-91J_LsJ-rQIsDTkTe9k2gn2Uu_DPKVONocHSQUiTRJxmkkbyzTNlFmALCF6Q23Bk9a1OwlViPthw/file# [following]
--2021-03-02 16:59:15--  https://ucf155bc8507c1c74e64add2c4df.dl.dropboxusercontent.com/cd/0/inline/BJ4FDzgPHcKz-Y5dVuMhX3RMv21LHZDbg-91J_LsJ-rQIsDTkTe9k2gn2Uu_DPKVONocHSQUiTRJxmkkbyzTNlFmALCF6Q23Bk9a1OwlViPthw/file
Resolving ucf155bc8

In [22]:
save_path = os.path.join(data_dir, "mask.tar.gz") 
# mask.tar.gzの解凍
with tarfile.open(save_path, 'r:*') as tar:
    tar.extractall(data_dir)

In [28]:
cd "/content/drive/My Drive/Colab Notebooks/pytorch_advanced/data"

/content/drive/My Drive/Colab Notebooks/pytorch_advanced/data


In [29]:
!cp -r "COCO.json" "cowboy-757575_640.jpg" "hit-1407826_640.jpg" "/content/data"

In [32]:
# パッケージのimport
import random
import math
import time
import pandas as pd
import numpy as np
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [33]:
# 初期設定
# Setup seeds
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)


# DataLoader作成

In [37]:
cd "/content"

/content


In [38]:
from utils.dataloader import make_datapath_list, DataTransform, COCOkeypointsDataset

# MS COCOのファイルパスリスト作成
train_img_list, train_mask_list, val_img_list, val_mask_list, train_meta_list, val_meta_list = make_datapath_list(
    rootpath="./data/")

# Dataset作成
# 本書ではデータ量の問題から、trainをval_listで作成している点に注意
train_dataset = COCOkeypointsDataset(
    val_img_list, val_mask_list, val_meta_list, phase="train", transform=DataTransform())

# 今回は簡易な学習とし検証データは作成しない
# val_dataset = CocokeypointsDataset(val_img_list, val_mask_list, val_meta_list, phase="val", transform=DataTransform())

# DataLoader作成
batch_size = 32

train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

# val_dataloader = data.DataLoader(
#    val_dataset, batch_size=batch_size, shuffle=False)

# 辞書型変数にまとめる
# dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}
dataloaders_dict = {"train": train_dataloader, "val": None}


# ネットワークモデル作成

In [39]:
from utils.openpose_net import OpenPoseNet
net = OpenPoseNet()


# 損失関数を定義

In [40]:
# 損失関数の設定
class OpenPoseLoss(nn.Module):
    """OpenPoseの損失関数のクラスです。"""

    def __init__(self):
        super(OpenPoseLoss, self).__init__()

    def forward(self, saved_for_loss, heatmap_target, heat_mask, paf_target, paf_mask):
        """
        損失関数の計算。

        Parameters
        ----------
        saved_for_loss : OpenPoseNetの出力(リスト)

        heatmap_target : [num_batch, 19, 46, 46]
            正解の部位のアノテーション情報

        heatmap_mask : [num_batch, 19, 46, 46]
            heatmap画像のmask

        paf_target : [num_batch, 38, 46, 46]
            正解のPAFのアノテーション情報

        paf_mask : [num_batch, 38, 46, 46]
            PAF画像のmask

        Returns
        -------
        loss : テンソル
            損失の値
        """

        total_loss = 0
        # ステージごとに計算します
        for j in range(6):

            # PAFsとheatmapsにおいて、マスクされている部分（paf_mask=0など）は無視させる
            # PAFs
            pred1 = saved_for_loss[2 * j] * paf_mask
            gt1 = paf_target.float() * paf_mask

            # heatmaps
            pred2 = saved_for_loss[2 * j + 1] * heat_mask
            gt2 = heatmap_target.float()*heat_mask

            total_loss += F.mse_loss(pred1, gt1, reduction='mean') + \
                F.mse_loss(pred2, gt2, reduction='mean')

        return total_loss


criterion = OpenPoseLoss()


# 最適化手法を設定

In [41]:
optimizer = optim.SGD(net.parameters(), lr=1e-2,
                      momentum=0.9,
                      weight_decay=0.0001)


# 学習を実施

In [42]:
# モデルを学習させる関数を作成


def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # 画像の枚数
    num_train_imgs = len(dataloaders_dict["train"].dataset)
    batch_size = dataloaders_dict["train"].batch_size

    # イテレーションカウンタをセット
    iteration = 1

    # epochのループ
    for epoch in range(num_epochs):

        # 開始時刻を保存
        t_epoch_start = time.time()
        t_iter_start = time.time()
        epoch_train_loss = 0.0  # epochの損失和
        epoch_val_loss = 0.0  # epochの損失和

        print('-------------')
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
                optimizer.zero_grad()
                print('（train）')

            # 今回は検証はスキップ
            else:
                continue
                # net.eval()   # モデルを検証モードに
                # print('-------------')
                # print('（val）')

            # データローダーからminibatchずつ取り出すループ
            for imges, heatmap_target, heat_mask, paf_target, paf_mask in dataloaders_dict[phase]:
                # ミニバッチがサイズが1だと、バッチノーマライゼーションでエラーになるのでさける
                if imges.size()[0] == 1:
                    continue

                # GPUが使えるならGPUにデータを送る
                imges = imges.to(device)
                heatmap_target = heatmap_target.to(device)
                heat_mask = heat_mask.to(device)
                paf_target = paf_target.to(device)
                paf_mask = paf_mask.to(device)

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):
                    # (out6_1, out6_2)は使わないので _ で代替
                    _, saved_for_loss = net(imges)

                    loss = criterion(saved_for_loss, heatmap_target,
                                     heat_mask, paf_target, paf_mask)
                    del saved_for_loss
                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                        if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            print('イテレーション {} || Loss: {:.4f} || 10iter: {:.4f} sec.'.format(
                                iteration, loss.item()/batch_size, duration))
                            t_iter_start = time.time()

                        epoch_train_loss += loss.item()
                        iteration += 1

                    # 検証時
                    # else:
                        #epoch_val_loss += loss.item()

        # epochのphaseごとのlossと正解率
        t_epoch_finish = time.time()
        print('-------------')
        print('epoch {} || Epoch_TRAIN_Loss:{:.4f} ||Epoch_VAL_Loss:{:.4f}'.format(
            epoch+1, epoch_train_loss/num_train_imgs, 0))
        print('timer:  {:.4f} sec.'.format(t_epoch_finish - t_epoch_start))
        t_epoch_start = time.time()

    # 最後のネットワークを保存する
    torch.save(net.state_dict(), 'weights/openpose_net_' +
               str(epoch+1) + '.pth')


In [43]:
# 学習・検証を実行する
num_epochs = 2
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)


使用デバイス： cuda:0
-------------
Epoch 1/2
-------------
（train）
イテレーション 10 || Loss: 0.0092 || 10iter: 64.3158 sec.
イテレーション 20 || Loss: 0.0081 || 10iter: 52.6837 sec.
イテレーション 30 || Loss: 0.0068 || 10iter: 53.0832 sec.
イテレーション 40 || Loss: 0.0061 || 10iter: 53.2376 sec.
イテレーション 50 || Loss: 0.0050 || 10iter: 55.4166 sec.
イテレーション 60 || Loss: 0.0041 || 10iter: 52.1875 sec.
イテレーション 70 || Loss: 0.0033 || 10iter: 54.2835 sec.
イテレーション 80 || Loss: 0.0035 || 10iter: 52.4707 sec.
イテレーション 90 || Loss: 0.0030 || 10iter: 51.4121 sec.
イテレーション 100 || Loss: 0.0026 || 10iter: 53.7975 sec.
イテレーション 110 || Loss: 0.0022 || 10iter: 53.3098 sec.
イテレーション 120 || Loss: 0.0021 || 10iter: 52.8239 sec.
イテレーション 130 || Loss: 0.0020 || 10iter: 51.3487 sec.
イテレーション 140 || Loss: 0.0021 || 10iter: 53.4716 sec.
イテレーション 150 || Loss: 0.0017 || 10iter: 52.1362 sec.
-------------
epoch 1 || Epoch_TRAIN_Loss:0.0043 ||Epoch_VAL_Loss:0.0000
timer:  835.7236 sec.
-------------
Epoch 2/2
-------------
（train）
イテレーション 160 || Loss: 0.0018

FileNotFoundError: ignored

以上